In [1]:
import pandas as pd
import numpy as np
import optuna
from optuna.samplers import TPESampler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_absolute_error, mean_squared_error
from sklearn.pipeline import Pipeline
from catboost import CatBoostRegressor
from importlib import reload
import logging
reload(logging)
import funcs 
import pickle

In [2]:
# Задаем формат логирования
logging.basicConfig(
   format="%(levelname)s: %(asctime)s: %(message)s",
    level=logging.INFO
)

In [3]:
# Cоздаем лог-файл
logger = funcs.get_logger(path="logs/", file="model.log")

In [4]:
# Считаем файл-csv импортированный из БД мониторинга в датафрейм
file_name = 'api01.csv'
data = pd.read_csv(file_name, sep=' ')
logger.info(f"Data file: {file_name}")
logger.info(f"Data shape: {data.shape}")
logger.info(f"Data head: \n{data.head()}")

INFO: 2024-09-11 06:11:57,462: Data file: api01.csv
INFO: 2024-09-11 06:11:57,467: Data shape: (40109, 4)
INFO: 2024-09-11 06:11:57,479: Data head: 
         date      time          id  value
0  12.08.2024  14:32:24  1723462344    912
1  12.08.2024  14:30:40  1723462240    657
2  12.08.2024  14:29:17  1723462157    872
3  12.08.2024  14:28:14  1723462094    500
4  12.08.2024  14:27:04  1723462024    852


In [5]:
# Получим выборку с очищенными данными
data_cleaned = funcs.get_data_cleaned(data, logger=logger)

INFO: 2024-09-11 06:12:01,997: Number of entries before filling in gaps: 1440
INFO: 2024-09-11 06:12:02,015: Number of entries after filling in the blanks: 1440
INFO: 2024-09-11 06:12:02,036: Basic statistical characteristics of end-to-end values of a time series: 
        lower_bound       median  upper_bound
count  1440.000000  1440.000000  1440.000000
mean    300.369132   343.182986   478.380243
std      11.924069    31.102221   139.257811
min     255.800000   287.000000   314.850000
25%     293.000000   322.000000   368.000000
50%     301.000000   336.500000   429.875000
75%     307.900000   357.500000   559.625000
max     351.050000   511.000000  1541.250000
INFO: 2024-09-11 06:12:03,994: Transformed data: 
                       y
date_time               
2024-07-13 14:33:00  785
2024-07-13 14:34:00  547
2024-07-13 14:35:00  448
2024-07-13 14:36:00  415
2024-07-13 14:38:00  381
INFO: 2024-09-11 06:12:03,997: Number of records before filling gaps and merging duplicates: 40109
INFO

In [6]:
# Выделям исходные признаки и целевой признак в отдельные наборы данных
X = data_cleaned.drop('y', axis=1)
y = data_cleaned.y

In [8]:
# Трансформируем исходные данные 
X_transformed = funcs.get_data_transformed(X)
logger.info('Shape before transform: {}'.format(X.shape))
logger.info('Shape after transform: {}'.format(X_transformed.shape))

INFO: 2024-09-11 06:12:42,177: Shape before transform: (43200, 0)
INFO: 2024-09-11 06:12:42,180: Shape after transform: (43200, 85)


In [9]:
# Вычислим момент времени для разделения датачета на тренировочную и тестовую выборки
split_datetime = X_transformed.index.max() - pd.to_timedelta(1, 'd')
# Создадим тренировочную и тестовую выборки
X_train = X_transformed[X_transformed.index<=split_datetime]
y_train = y[y.index<=split_datetime]
X_test = X_transformed[X_transformed.index>split_datetime]
y_test = y[y.index>split_datetime]
logger.info(f'Training sample size: {X_train.shape}, begin: {X_train.index.min()}, end: {X_train.index.max()}')
logger.info(f'Test sample size: {X_test.shape}, begin: {X_test.index.min()}, end: {X_test.index.max()}')

INFO: 2024-09-11 06:12:49,181: Training sample size: (41760, 85), begin: 2024-07-13 14:33:00, end: 2024-08-11 14:32:00
INFO: 2024-09-11 06:12:49,186: Test sample size: (1440, 85), begin: 2024-08-11 14:33:00, end: 2024-08-12 14:32:00


In [10]:
# Выполним масштабирование исходных признаков
scaler = MinMaxScaler()
# Подгоняем параметры стандартизатора
scaler.fit(X_train)
# Производим стандартизацию тренировочной выборки
X_train_scaled = pd.DataFrame(scaler.transform(X_train),
                              columns=X_train.columns, index=X_train.index)
# Производим стандартизацию тестовой выборки
X_test_scaled = pd.DataFrame(scaler.transform(X_test),
                              columns=X_test.columns, index=X_test.index)

In [12]:
# Объявим вспомогательную функцию по валидации данных временного ряда.
def timeseries_validation(model, X_train, y_train, n_splits=3):
  tss = TimeSeriesSplit(n_splits=n_splits, test_size=1440)
  train_test_groups = tss.split(X_train)
  metrics = []
  for train_index, test_index in train_test_groups:
    # обучаем модель
    model.fit(X_train.iloc[train_index], y_train.iloc[train_index])
    metrics.append(mean_absolute_percentage_error(y_train.iloc[test_index],
                                          model.predict(X_train.iloc[test_index])))
  return np.mean(metrics)

In [13]:
# Объявим целевую функцию для поиска гиперпараметров, 
# происходит инициализация модели по входящему в функцию параметру, 
# далее выполняется валидация на основе обучающей выборки, и выдается средняя метрика.
def optuna_CatBoostRegressor(trial):
            # задаем пространства поиска гиперпараметров
            model_params =  {
                'n_estimators': trial.suggest_int('n_estimators', 100, 1500),
                #'learning_rate': trial.suggest_float("learning_rate", 0.001, 0.2, log=True),
                "depth": trial.suggest_int("depth", 1, 12),
                #"boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
                #"bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
                "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 2, 64),
                "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1, log=False),
                'random_state': trial.suggest_int("random_state", 42, 42),
                'verbose': trial.suggest_int("verbose", 0, 0)
            }
            # создаем модель
            model = CatBoostRegressor(**model_params)
            # запускаем валидацию обучающей выбоки
            score = timeseries_validation(model, X_train_scaled, y_train)
            return score

In [14]:
# Запустим поиск оптимальных гиперпараметров модели
sampler = TPESampler(seed=42)
optuna.logging.set_verbosity(optuna.logging.WARNING)
print('Поиск оптимальных гиперпараметров: ', 'CatBoostRegressor')
study = optuna.create_study(direction='minimize', study_name='CatBoostRegressor', sampler=sampler)
study.optimize(optuna_CatBoostRegressor, n_trials=1)
# Сохраним оптимальные гиперпараметры модели в структуру моделей
best_params = study.best_trial.params
logger.info(f'Optimal hyperparameters for the CatBoostRegressor model found: {best_params}')

[I 2024-09-11 06:13:11,633] A new study created in memory with name: CatBoostRegressor


Поиск оптимальных гиперпараметров:  CatBoostRegressor


[I 2024-09-11 06:14:37,888] Trial 0 finished with value: 0.06928951915823804 and parameters: {'n_estimators': 624, 'depth': 12, 'min_data_in_leaf': 48, 'colsample_bylevel': 0.0638792635777333, 'random_state': 42, 'verbose': 0}. Best is trial 0 with value: 0.06928951915823804.
INFO: 2024-09-11 06:14:37,892: Optimal hyperparameters for the CatBoostRegressor model found: {'n_estimators': 624, 'depth': 12, 'min_data_in_leaf': 48, 'colsample_bylevel': 0.0638792635777333, 'random_state': 42, 'verbose': 0}


In [15]:
# Создаём пайплайн, который включает нормализацию, отбор признаков и обучение модели
pipe = Pipeline([ 
  ('Scaling', MinMaxScaler()),
  ('CatBoostRegressor', CatBoostRegressor(**best_params))
  ])
# Обучаем пайплайн
pipe.fit(X_train, y_train);

In [16]:
# Объявим функцию по выводу метрик
def get_metrics(data_true, data_pred):
  metric_funcs = {r2_score: 'R2', mean_absolute_percentage_error: 'MAPE',
                  mean_absolute_error: 'MAE', mean_squared_error: 'MSE'}
  return [(metric_funcs[func], func(data_true, data_pred)) for func in metric_funcs]

In [17]:
# Сделаем предсказание обучающей выборки
y_pred_train = pipe.predict(X_train)
# Сохраним обучающие метрики в таблице итоговых результатов
metrics = get_metrics(y_train, y_pred_train)
logger.info(f"Learning metrics: \n{metrics}")
# Сделаем предсказание тестовой выборки
y_pred_test = pipe.predict(X_test)
# Сохраним тестовые метрики в таблице итоговых результатов
metrics = get_metrics(y_test, y_pred_test)
logger.info(f"Test metrics: \n{metrics}")

INFO: 2024-09-11 06:15:20,278: Learning metrics: 
[('R2', 0.4932229687487738), ('MAPE', 0.059861532568902705), ('MAE', 22.904933627853936), ('MSE', 1599.6877476900324)]
INFO: 2024-09-11 06:15:20,353: Test metrics: 
[('R2', 0.23206125245924125), ('MAPE', 0.05930073315435337), ('MAE', 24.621374863129024), ('MSE', 2836.484668333923)]


In [22]:
y_pred_test.sum()

483097.7193900346

In [22]:
# Сериализуем pipeline и записываем результат в файл
file_name = 'pipeline.pkl'
with open(file_name, 'wb') as output:
    pickle.dump(pipe, output)
logger.info(f'The serialized pipeline is written to a file: {file_name}')

INFO: 2024-09-11 05:27:22,091: The serialized pipeline is written to a file: pipeline.pkl


In [20]:
# Десериализуем pipeline из файла
with open('pipeline.pkl', 'rb') as pkl_file:
    loaded_pipe = pickle.load(pkl_file)